In [1]:
import os
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import transformers

# Если используете albumentations, раскомментируйте импорт
# !pip install albumentations>=1.0.0 --quiet
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
from transformers import SegformerImageProcessor, SegformerForSemanticSegmentation

rescale = 1
os.environ["CUDA_VISIBLE_DEVICES"] = "4,6"
CUSTOM_MEAN = [0.1778, 0.2696, 0.1686]
CUSTOM_STD = [0.0942, 0.0915, 0.0762]
PATCH_SIZE = 256 // rescale
NUM_LABELS = 3

/home/neos/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:albumentations.check_version:A new version of Albumentations is available: 2.0.5 (you have 1.4.7). Upgrade using: pip install --upgrade albumentations


In [2]:
from datasets import load_dataset
import logging

logging.basicConfig(level=logging.DEBUG)

dataset = load_dataset("./my_segmentation_dataset.py", data_dir="./outputs_sr/data_sr_TTST" + f"_x{rescale}" * (rescale != 1), trust_remote_code=True)
train_ds = dataset["train"]
# valid_ds = dataset["validation"]
valid_ds = load_dataset("./my_segmentation_dataset.py", data_dir="./outputs_sr/data_sr_TTST" + f"_x{rescale}" * (rescale != 1), trust_remote_code=True)["validation"]

id2label = {0: "unlabeled", 1: "forest0", 2: "forest1"}
label2id = {v: k for k, v in id2label.items()}

INFO:datasets:PyTorch version 2.6.0+cu124 available.


In [3]:
def transforms(examples):
    transformA = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        
        # При необходимости можно добавить и другие аугментации, 
        # например, яркость/контраст, шум и т.д.
        
        # Простейшая нормализация, если нужно
        # A.Normalize(mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD),  
        # ToTensorV2()
    ])
    
    transformed_images, transformed_masks = [], []
    for image, seg_mask in zip(examples["image"], examples["annotation"]):
        image, seg_mask = np.array(image), np.array(seg_mask)
        transformed = transformA(image=image, mask=seg_mask)
        transformed_images.append(transformed["image"])
        transformed_masks.append(transformed["mask"])
    examples["image"] = transformed_images
    examples["annotation"] = transformed_masks
    return examples

In [4]:
import evaluate

metric = evaluate.load("mean_iou")
metric_f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    with torch.no_grad():
        # print(eval_pred.losses)
        logits, labels = eval_pred

        pred_labels = np.argmax(logits, axis=1)
        metrics = metric.compute(
            predictions=pred_labels,
            references=labels,
            num_labels=NUM_LABELS,
            ignore_index=255,
            reduce_labels=False,
        )
        metrics_f1 = metric_f1.compute(
            predictions=pred_labels.flatten(),
            references=labels.flatten(),
            average="macro",
            # ignore_index=255,
            # reduce_labels=True,
        )
        per_category_accuracy = metrics.pop("per_category_accuracy").tolist()
        per_category_iou = metrics.pop("per_category_iou").tolist()
    
        metrics.update({f"accuracy_{id2label[i]}": v for i, v in enumerate(per_category_accuracy)})
        metrics.update({f"iou_{id2label[i]}": v for i, v in enumerate(per_category_iou)})
        metrics.update({"f1_score": metrics_f1["f1"]})
    
        return metrics

In [5]:
from transformers import AutoImageProcessor

# checkpoint = "nvidia/mit-b0"
checkpoint = "nvidia/segformer-b0-finetuned-ade-512-512"
# checkpoint = "sawthiha/segformer-b0-finetuned-deprem-satellite"
# checkpoint = "nvidia/segformer-b5-finetuned-ade-640-640"
# checkpoint = "facebook/maskformer-swin-base-ade"
# checkpoint = "nvidia/segformer-b3-finetuned-cityscapes-1024-1024"
# image_processor = SegformerImageProcessor.from_pretrained(checkpoint, size={"height": PATCH_SIZE, "width": PATCH_SIZE}, do_reduce_labels=False)
image_processor = AutoImageProcessor.from_pretrained(checkpoint, size={"height": PATCH_SIZE, "width": PATCH_SIZE},
                                                     do_reduce_labels=False, use_fast=True, image_mean=CUSTOM_MEAN, image_std=CUSTOM_STD)

from torchvision.transforms import ColorJitter

jitter = ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.1)

def train_transforms(example_batch):
    images = [jitter(x) for x in example_batch["image"]]
    example_batch = transforms(example_batch)
    images = [x for x in example_batch["image"]]
    labels = [x for x in example_batch["annotation"]]
    inputs = image_processor(images, labels)
    return inputs


def val_transforms(example_batch):
    images = [x for x in example_batch["image"]]
    labels = [x for x in example_batch["annotation"]]
    inputs = image_processor(images, labels)
    return inputs


train_ds.set_transform(train_transforms)
valid_ds.set_transform(val_transforms)

`use_fast` is set to `True` but the image processor class does not have a fast version.  Falling back to the slow version.


In [6]:
from transformers import TrainerCallback
import subprocess

class SaveEvalPredictionsCallback(TrainerCallback):
    def __init__(self, trainer, eval_dataset, output_dir, num_labels=3, val_transforms=val_transforms):
        super().__init__()
        self.trainer = trainer  # явно передаём объект trainer
        self.eval_dataset = eval_dataset
        self.output_dir = output_dir
        self.num_labels = num_labels
        os.makedirs(self.output_dir, exist_ok=True)

    def upvalue(self, img):
        return ((img / (self.num_labels - 1)) * 255).astype(np.uint8)

    def on_evaluate(self, args, state, control, **kwargs):
        # Выполняем предсказания на eval_dataset с помощью полученного trainer
        pred_res = self.trainer.predict(self.eval_dataset)
        pred_labels = np.argmax(pred_res.predictions, axis=1)  # [N, H, W]
        # upvalue
        pred_labels = self.upvalue(pred_labels)
    
        # Создаём поддиректорию в зависимости от шага
        step_folder = os.path.join(self.output_dir, f"step_{state.global_step}")
        os.makedirs(step_folder, exist_ok=True)
    
        # Сохраняем каждую маску как png
        try:
            self.eval_dataset.reset_format()
            for i, label_map in enumerate(pred_labels):
                label_img = Image.fromarray(np.concatenate((np.repeat(label_map[:, :, None], 3, -1), self.upvalue(np.repeat(np.array(self.eval_dataset[i]["annotation"])[:, :, None], 3, -1)), self.eval_dataset[i]["image"]), axis=1))
                label_img.save(os.path.join(step_folder, f"pred_{self.eval_dataset[i]['filename'].rsplit('.')[0]}.png"))
        except RuntimeError as e:
            self.eval_dataset.set_transform(val_transforms)
            raise e
        self.eval_dataset.set_transform(val_transforms)

        print(f"Saved evaluation predictions to: {step_folder}")

class ReconstructionCallback(TrainerCallback):
    """
    Колбэк, который после сохранения чекпоинта запускает сторонний скрипт
    для реконструкции изображений (или постобработки, измерения IoU и т.п.).
    """
    def on_save(self, args, state, control, **kwargs):
        """
        Вызывается сразу после сохранения модели.
        Параметры:
        -----------
        args  : TrainingArguments
        state : TrainerState
        control : TrainerControl
        """
        # Путь к файлу скрипта, который вы хотите запустить
        script_path = "reconstructionv3.py"
    
        # Пример: передадим в скрипт некоторые аргументы, например,
        # путь к только что сохранённому чекпоинту.
        # state.best_model_checkpoint или state.global_step и т.д. 
        # Можете скорректировать по желанию.
        checkpoint_dir = args.output_dir.rsplit('/', 1)[0]
        step_str = str(state.global_step - args.save_steps)
        
        if step_str == "0":
            return control
        
        # Вызов внешнего скрипта:
        try:
            cmd = [
                # "CUDA_VISIBLE_DEVICES=" + os.getenv("CUDA_VISIBLE_DEVICES"),
                "python3", script_path,
                "--model", "SegFormer",
                "--out_dir", checkpoint_dir,
                "--step", step_str,
                "--rescale", str(rescale),
                "--format", "png"
            ]
            pipe = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, bufsize=1, text=True)
            while pipe.poll() is None:
                continue
                print(pipe.stdout.readline().strip())
            if pipe.returncode != 0:
                print('Command exited with non zero return code')
                print('Return code = %d', pipe.returncode)
                raise RuntimeError()
            # print('The job finished successfully')
            print(f"[INFO] Reconstruction script {script_path} завершился успешно.")
        except subprocess.CalledProcessError as e:
            print(f"[ERROR] Что-то пошло не так при вызове {script_path}. Код возврата: {e.returncode}")
            # При необходимости можно выбросить исключение или продолжить
    
        return control

In [7]:
from transformers import (
    AutoModelForSemanticSegmentation,
    TrainingArguments,
    Trainer
)
from transformers.modeling_outputs import SemanticSegmenterOutput
from typing import Optional, Union, Tuple

model = SegformerForSemanticSegmentation.from_pretrained(checkpoint, num_labels=3, ignore_mismatched_sizes=True)

class DiceLoss(nn.Module):
    def __init__(self, smooth=1.0):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, outputs, targets):
        """
        Args:
            outputs (torch.Tensor): Прогнозы модели размерности (N, C, H, W), от LogSoftmax или Softmax.
            targets (torch.Tensor): Истинные метки размерности (N, H, W) или (N, 1, H, W) с классами в [0, C-1].

        Returns:
            torch.Tensor: Значение Dice Loss.
        """
        num_classes = outputs.size(1)

        # Конвертируем targets в one-hot формат, если необходимо
        if targets.ndim == 3:
            targets = targets.unsqueeze(1)
        targets_one_hot = torch.zeros_like(outputs).scatter_(1, targets, 1)
        weights = outputs.new([1., 1., 2.])

        # Вычисляем Dice Loss для каждого класса
        dice_loss = 0.0
        for c in range(num_classes):
            output_c = outputs[:, c, :, :]
            target_c = targets_one_hot[:, c, :, :]
            
            intersection = (output_c * target_c).sum()
            union = output_c.sum() + target_c.sum() - intersection

            dice_loss += (intersection + self.smooth) / (union + self.smooth) * weights[c]

        return 1 - dice_loss / num_classes

class ModelWrapper(torch.nn.Module):
    def __init__(self, segformer, patch_size=PATCH_SIZE, accepts_loss_kwargs=False):
        super().__init__()
        self.main = segformer
        self.accepts_loss_kwargs = accepts_loss_kwargs
        self.patch_size = patch_size
        self.dice_loss = DiceLoss()

    # def forward(self, *args, **kwargs):
    #     outputs = self.main(*args, **kwargs)
    def forward(
        self,
        pixel_values: torch.FloatTensor,
        labels: Optional[torch.LongTensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple, SemanticSegmenterOutput]:
        outputs = self.main(pixel_values, labels=labels, output_attentions=output_attentions,
                            output_hidden_states=output_hidden_states, return_dict=return_dict)
        loss, logits_val = outputs.loss, outputs.logits
        upsampled_val_logits = torch.nn.functional.interpolate(
        logits_val, size=(self.patch_size, self.patch_size), mode="bilinear", align_corners=False
        )
        loss = F.cross_entropy(upsampled_val_logits, labels, weight=upsampled_val_logits.new([1., 1., 2.]))
        loss += 0.2 * self.dice_loss(upsampled_val_logits, labels)
        return SemanticSegmenterOutput(loss=loss, logits=upsampled_val_logits)

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b0-finetuned-ade-512-512 and are newly initialized because the shapes did not match:
- decode_head.classifier.bias: found shape torch.Size([150]) in the checkpoint and torch.Size([3]) in the model instantiated
- decode_head.classifier.weight: found shape torch.Size([150, 256, 1, 1]) in the checkpoint and torch.Size([3, 256, 1, 1]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
os.environ["WANDB_LOG_MODEL"] = "checkpoint"
os.environ["WANDB_WATCH"] = "gradients"
# os.environ["WANDB_DISABLED"] = "true"
os.environ["HF_TOKEN"] = "<your-token>"
# os.environ["WANDB_RESUME"] = "must"
# os.environ["WANDB_RUN_ID"] = "gqg8gx05"

In [9]:
import warnings
warnings.filterwarnings("ignore")

modelwr = ModelWrapper(model)
# modelwr = transformers.MaskFormerForInstanceSegmentation.from_pretrained(checkpoint, num_labels=NUM_LABELS - 1, ignore_mismatched_sizes=True)
print(f"num_model_parameters:{sum(par.numel() for par in modelwr.parameters())}")
outputs_dir = "outputs_sr_TTST"
if rescale > 1:
    outputs_dir += f"_x{rescale}"

training_args = TrainingArguments(
    output_dir=os.path.join(outputs_dir, "checkpoints"),
    learning_rate=6e-5,
    num_train_epochs=1000,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    save_total_limit=5,
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=2000,
    eval_steps=500,
    logging_steps=500,
    eval_accumulation_steps=5,
    remove_unused_columns=False,
    load_best_model_at_end=True,
    metric_for_best_model = "eval_mean_iou",
    # log_level='debug',
    # use_cpu=True,
    # push_to_hub=True,
)

trainer = Trainer(
    model=modelwr,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    compute_metrics=compute_metrics,
)
# callback = SaveEvalPredictionsCallback(
#     trainer=trainer,
#     eval_dataset=valid_ds,
#     output_dir=os.path.join(outputs_dir, "eval_predict"),
# )
rec_callback = ReconstructionCallback()
# добавляем callback после инициализации trainer'а
trainer.add_callback(rec_callback)
trainer.train()
# trainer.train(resume_from_checkpoint=True)

num_model_parameters:3714915


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: arslan-valeev-03 (course_sr) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Mean Iou,Mean Accuracy,Overall Accuracy,Accuracy Unlabeled,Accuracy Forest0,Accuracy Forest1,Iou Unlabeled,Iou Forest0,Iou Forest1,F1 Score
500,0.781700,1.339200,0.371491,0.653190,0.367397,0.893744,0.072368,0.993458,0.787541,0.071930,0.255002,0.473909
1000,0.946300,2.519032,0.541503,0.769036,0.616636,0.868571,0.453678,0.984859,0.816865,0.444904,0.362740,0.682465
1500,0.884700,0.878850,0.773070,0.894179,0.889200,0.954093,0.894981,0.833462,0.789225,0.844907,0.685077,0.870414
2000,0.942400,0.377941,0.711959,0.831828,0.849669,0.913816,0.887095,0.694574,0.791656,0.798770,0.545452,0.825908
2500,0.659600,0.671462,0.737337,0.816982,0.883918,0.697185,0.940582,0.813179,0.693325,0.852697,0.665989,0.846299
3000,1.044700,0.240015,0.776746,0.838365,0.896990,0.846040,0.968683,0.700374,0.821409,0.863235,0.645594,0.871060
3500,0.806100,8.592673,0.776209,0.900251,0.879140,0.931748,0.860037,0.908966,0.849745,0.829377,0.649504,0.871005
4000,0.679900,0.859046,0.814760,0.896822,0.910371,0.917747,0.930693,0.842024,0.858328,0.875471,0.710480,0.896034
4500,0.507000,0.749869,0.704713,0.873010,0.808919,0.903408,0.738144,0.977478,0.851290,0.731027,0.531821,0.819551
5000,0.603800,0.822231,0.776823,0.855594,0.891633,0.871823,0.937962,0.756996,0.840215,0.858093,0.632161,0.870476


wandb: Adding directory to artifact (./outputs_sr_TTST/checkpoints/checkpoint-2000)... Done. 0.1s
wandb: Adding directory to artifact (./outputs_sr_TTST/checkpoints/checkpoint-4000)... Done. 0.1s


[INFO] Reconstruction script reconstructionv3.py завершился успешно.


...

wandb: Adding directory to artifact (./outputs_sr_TTST/checkpoints/checkpoint-8000)... Done. 0.1s


[INFO] Reconstruction script reconstructionv3.py завершился успешно.


wandb: Adding directory to artifact (./outputs_sr_TTST/checkpoints/checkpoint-10000)... Done. 0.1s


[INFO] Reconstruction script reconstructionv3.py завершился успешно.


KeyboardInterrupt: 